In [52]:
# 모듈 import

from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.utils import get_device
from pprint import pprint

In [53]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
cfg = Config.fromfile('./configs/_jsh_custom_configs/custum_faster_rcnn_effb3_fpn_1x_coco.py') # config 위치 설정
root='../../folded_anns/' # data root 상대 경로 설정

# dataset config 수정
cfg.data.train.classes = classes
cfg.data.train.img_prefix = root
cfg.data.train.ann_file = root + 'train_2.json' # train json 정보
cfg.data.train.pipeline[2]['img_scale'] = (512,512) # Resize

cfg.data.val.classes = classes
cfg.data.val.img_prefix = root
cfg.data.val.ann_file = root + 'val_2.json' # val json 정보
cfg.data.val.pipeline[1]['img_scale'] = (512,512) # Resize


cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json' # test json 정보
cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize

cfg.data.samples_per_gpu = 4

cfg.seed = 2022
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/custom_faster_rcnn_effb3_fpn_1x_trash'

cfg.model.roi_head.bbox_head.num_classes = 10

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.checkpoint_config = dict(max_keep_ckpts=3, interval=1)
cfg.device = get_device()

In [54]:
# check config
pprint(dict(cfg))

{'auto_scale_lr': {'base_batch_size': 16, 'enable': False},
 'checkpoint': 'https://download.openmmlab.com/mmclassification/v0/efficientnet/efficientnet-b3_3rdparty_8xb32-aa_in1k_20220119-5b4887a0.pth',
 'checkpoint_config': {'interval': 1, 'max_keep_ckpts': 3},
 'classes': ('General trash',
             'Paper',
             'Paper pack',
             'Metal',
             'Glass',
             'Plastic',
             'Styrofoam',
             'Plastic bag',
             'Battery',
             'Clothing'),
 'cudnn_benchmark': True,
 'custom_hooks': [{'type': 'NumClassCheckHook'}],
 'data': {'samples_per_gpu': 4,
          'test': {'ann_file': '../../dataset/test.json',
                   'classes': ('General trash',
                               'Paper',
                               'Paper pack',
                               'Metal',
                               'Glass',
                               'Plastic',
                               'Styrofoam',
                     

In [55]:
# build_dataset
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=0.06s)
creating index...
index created!


In [56]:
# dataset 확인
datasets[0]


CocoDataset Train dataset with number of images 3901, and instance counts: 
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| category          | count | category      | count | category        | count | category    | count | category     | count |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| 0 [General trash] | 3224  | 1 [Paper]     | 5103  | 2 [Paper pack]  | 714   | 3 [Metal]   | 761   | 4 [Glass]    | 770   |
| 5 [Plastic]       | 2385  | 6 [Styrofoam] | 958   | 7 [Plastic bag] | 4225  | 8 [Battery] | 128   | 9 [Clothing] | 362   |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+

In [57]:
# 모델 build 및 pretrained network 불러오기
model = build_detector(cfg.model)
model.init_weights()

2022-11-24 19:14:28,204 - mmcv - INFO - initialize EfficientNet with init_cfg {'type': 'Pretrained', 'prefix': 'backbone', 'checkpoint': 'https://download.openmmlab.com/mmclassification/v0/efficientnet/efficientnet-b3_3rdparty_8xb32-aa_in1k_20220119-5b4887a0.pth'}
2022-11-24 19:14:28,205 - mmcv - INFO - load backbone in model from: https://download.openmmlab.com/mmclassification/v0/efficientnet/efficientnet-b3_3rdparty_8xb32-aa_in1k_20220119-5b4887a0.pth
2022-11-24 19:14:28,335 - mmcv - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: layers.6.bn.running_var, layers.6.bn.running_mean, layers.6.bn.weight, layers.6.bn.bias, layers.6.conv.weight

2022-11-24 19:14:28,378 - mmcv - INFO - initialize FPN with init_cfg {'type': 'Xavier', 'layer': 'Conv2d', 'distribution': 'uniform'}
2022-11-24 19:14:28,397 - mmcv - INFO - initialize RPNHead with init_cfg {'type': 'Normal', 'layer': 'Conv2d', 'std': 0.01}


load checkpoint from http path: https://download.openmmlab.com/mmclassification/v0/efficientnet/efficientnet-b3_3rdparty_8xb32-aa_in1k_20220119-5b4887a0.pth


2022-11-24 19:14:28,409 - mmcv - INFO - initialize Shared2FCBBoxHead with init_cfg [{'type': 'Normal', 'std': 0.01, 'override': {'name': 'fc_cls'}}, {'type': 'Normal', 'std': 0.001, 'override': {'name': 'fc_reg'}}, {'type': 'Xavier', 'distribution': 'uniform', 'override': [{'name': 'shared_fcs'}, {'name': 'cls_fcs'}, {'name': 'reg_fcs'}]}]
2022-11-24 19:14:28,516 - mmcv - INFO - 
backbone.layers.0.conv.weight - torch.Size([40, 3, 3, 3]): 
PretrainedInit: load from https://download.openmmlab.com/mmclassification/v0/efficientnet/efficientnet-b3_3rdparty_8xb32-aa_in1k_20220119-5b4887a0.pth 
 
2022-11-24 19:14:28,518 - mmcv - INFO - 
backbone.layers.0.bn.weight - torch.Size([40]): 
PretrainedInit: load from https://download.openmmlab.com/mmclassification/v0/efficientnet/efficientnet-b3_3rdparty_8xb32-aa_in1k_20220119-5b4887a0.pth 
 
2022-11-24 19:14:28,518 - mmcv - INFO - 
backbone.layers.0.bn.bias - torch.Size([40]): 
PretrainedInit: load from https://download.openmmlab.com/mmclassificati

In [58]:
# 모델 학습
train_detector(model, datasets[0], cfg, distributed=False, validate=True)

2022-11-24 19:14:31,534 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
2022-11-24 19:14:31,563 - mmdet - INFO - Start running, host: root@69f5c3117c58, work_dir: /opt/ml/baseline/mmdetection/work_dirs/custom_faster_rcnn_effb3_fpn_1x_trash
2022-11-24 19:14:31,564 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_iter:
(VERY_HIGH   ) StepLrUpdaterHook                  
(LOW         ) IterTimerHook         

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


2022-11-24 19:14:46,516 - mmdet - INFO - Epoch [1][50/976]	lr: 1.978e-03, eta: 0:58:05, time: 0.299, data_time: 0.050, memory: 5426, loss_rpn_cls: 0.2833, loss_rpn_bbox: 0.0242, loss_cls: 0.9446, acc: 87.9697, loss_bbox: 0.0521, loss: 1.3043, grad_norm: 15.1810
2022-11-24 19:14:59,189 - mmdet - INFO - Epoch [1][100/976]	lr: 3.976e-03, eta: 0:53:26, time: 0.253, data_time: 0.006, memory: 5426, loss_rpn_cls: 0.0936, loss_rpn_bbox: 0.0241, loss_cls: 0.1673, acc: 97.7705, loss_bbox: 0.0662, loss: 0.3513, grad_norm: 1.4147
2022-11-24 19:15:11,810 - mmdet - INFO - Epoch [1][150/976]	lr: 5.974e-03, eta: 0:51:41, time: 0.252, data_time: 0.006, memory: 5426, loss_rpn_cls: 0.0735, loss_rpn_bbox: 0.0171, loss_cls: 0.1188, acc: 97.9629, loss_bbox: 0.0556, loss: 0.2650, grad_norm: 0.9139
2022-11-24 19:15:24,426 - mmdet - INFO - Epoch [1][200/976]	lr: 7.972e-03, eta: 0:50:42, time: 0.252, data_time: 0.006, memory: 5426, loss_rpn_cls: 0.0782, loss_rpn_bbox: 0.0184, loss_cls: 0.1288, acc: 97.7939, los

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 982/982, 19.3 task/s, elapsed: 51s, ETA:     0s

2022-11-24 19:19:33,566 - mmdet - INFO - Evaluating bbox...


Loading and preparing results...
DONE (t=0.33s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=3.51s).
Accumulating evaluation results...


2022-11-24 19:19:38,994 - mmdet - INFO - 
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.002
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.010
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.003
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.017
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.017
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.017
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=1000

DONE (t=1.39s).


2022-11-24 19:19:54,532 - mmdet - INFO - Epoch [2][50/976]	lr: 2.000e-02, eta: 0:44:49, time: 0.309, data_time: 0.051, memory: 5426, loss_rpn_cls: 0.0714, loss_rpn_bbox: 0.0221, loss_cls: 0.1305, acc: 97.4746, loss_bbox: 0.0723, loss: 0.2963, grad_norm: 0.8961
2022-11-24 19:20:07,359 - mmdet - INFO - Epoch [2][100/976]	lr: 2.000e-02, eta: 0:44:39, time: 0.257, data_time: 0.006, memory: 5426, loss_rpn_cls: 0.0684, loss_rpn_bbox: 0.0200, loss_cls: 0.1443, acc: 97.1826, loss_bbox: 0.0807, loss: 0.3135, grad_norm: 0.9170
2022-11-24 19:20:20,216 - mmdet - INFO - Epoch [2][150/976]	lr: 2.000e-02, eta: 0:44:28, time: 0.257, data_time: 0.006, memory: 5426, loss_rpn_cls: 0.0857, loss_rpn_bbox: 0.0223, loss_cls: 0.1472, acc: 97.0342, loss_bbox: 0.0857, loss: 0.3409, grad_norm: 1.0217
2022-11-24 19:20:32,919 - mmdet - INFO - Epoch [2][200/976]	lr: 2.000e-02, eta: 0:44:17, time: 0.254, data_time: 0.006, memory: 5426, loss_rpn_cls: 0.0691, loss_rpn_bbox: 0.0200, loss_cls: 0.1389, acc: 97.2441, loss